# Infer ***soma*** -  1️⃣ 

> WARNING: (🚨🚨🚨🚨 Steps 3-9 depend on establishing a good solution here.)


>> WARNING:  THIS DOES NOT WORK WELL - lacking fluorescent marker
>> #### Because we do NOT have a direct cell membrane / soma signal, this segmentation is trickiest and potentially most problematic part of the overall sub-cellular component inference pipeline. We are using the nuclei of the cell with the brightest total fluorescence (all channels) to identify a single soma for all downstream steps. The Soma (via the Cytosol mask) will be used to define ALL subsequent sub-cellular Objects.

--------------

## OBJECTIVE: 
### ✅ Infer sub-cellular component #2: ***soma***/cell body in order to understand interactome 

Infer a segmentation of the cell body -- the ***soma*** -- in order to measure its shape, position, and size.

CONTEXT: "Soma" is used here becuase subsequent experiments will contain neurons who's soma has a similar shape to an iPS cell body.

## OVERVIEW: 

This method is used in the case where there is no cell fill/membrane marker.

We will infer the soma from a combination of fluorescent signals. The current selection includes the lysosomes, ER, and Golgi (e.g., 'Ch = 1, 3, 5) which have some intracellular fluorescence - likely from off target marker localization to the entire soma and/or the cell membrane. There are two other channels, the residual channel from linear unmixing (e.g. `ch = 7`) and the lipid droplet channel (e.g., `ch = 6`) that could more lead to more unbiased selection of the entire soma. However, the drawback of these two markers is that they are present in every cell which makes downstream cell selection more challenging. To expand on this, we will be collecting per cell measurements, so each cell area has to be segmented individually even if there are two appropriately labeled cells within one field of view. 

## preamble

1. imports
2. setup
4. infer-soma
    * input
    * pre-processing
    * core processing
    * post-processing
    * select individual cell
    * output



## IMPORTS

In [1]:
# top level imports
from pathlib import Path
import os, sys
from collections import defaultdict

import numpy as np

from scipy import ndimage as ndi
from aicssegmentation.core.pre_processing_utils import ( intensity_normalization, 
                                                         image_smoothing_gaussian_slice_by_slice )
from aicssegmentation.core.MO_threshold import MO
from aicssegmentation.core.utils import hole_filling

from skimage import filters, morphology
from skimage.segmentation import watershed
from skimage.morphology import remove_small_holes   # function for post-processing (size filter)
from skimage.measure import label
import skimage

# # package for io 
from aicsimageio import AICSImage

import napari

### import local python functions in ../infer_subc_2d
sys.path.append(os.path.abspath((os.path.join(os.getcwd(), '..'))))


from infer_subc_2d.utils.file_io import (read_czi_image,
                                         list_image_files)

from infer_subc_2d.constants import (TEST_IMG_N,
                                                                    NUC_CH ,
                                                                    LYSO_CH ,
                                                                    MITO_CH ,
                                                                    GOLGI_CH ,
                                                                    PEROXI_CH ,
                                                                    ER_CH ,
                                                                    LIPID_CH ,
                                                                    RESIDUAL_CH )                                                                    
from infer_subc_2d.utils.img import *

from infer_subc_2d.organelles import fixed_get_optimal_Z_image, fixed_find_optimal_Z, find_optimal_Z

%load_ext autoreload
%autoreload 2

## SETUP
### Get and load Image for processing - specifically for __raw multichannel__ image (.czi format)
CUSTOMIZE WITH: 
1. updated path to data
2. updated folder name for "raw" data

> NOTE: we are operating on a single "test" image in this notebook.  The batch-processing of all the images will be happen at the end of the notebook after we have developed/confirmed the setmentation procedures and parameter settings.

In [ ]:
# this will be the example image for testing the pipeline below
test_img_n = TEST_IMG_N

# build the datapath
# all the imaging data goes here.
# CUSTOMIZE HERE --->
data_root_path = Path(os.path.expanduser("~")) / "Documents/Python Scripts/Infer-subc-2D"

# linearly unmixed ".czi" files are here
# CUSTOMIZE HERE --->
data_path = data_root_path / "raw"
im_type = ".czi"

# get the list of all files
img_file_list = list_image_files(data_path,im_type)
test_img_name = img_file_list[test_img_n]


In [ ]:
# isolate image as an ndarray and metadata as a dictionary
img_data,meta_dict = read_czi_image(test_img_name)

# get some top-level info about the RAW data
channel_names = meta_dict['name']
img = meta_dict['metadata']['aicsimage']
scale = meta_dict['scale']
channel_axis = meta_dict['channel_axis']


### Get and load Image for processing - specifically for __pre-processed__ images (.OME TIF format)

> #### Preprocessing:
> 
> In this instance, we are using [Huygens Essential Software](https://svi.nl/Homepage) to deconvolve 3D fluorescence confocal images. The output is one 3-dimensional .tif file for each channel in the original image.

The basic steps here include:
1. creating a separate list of image names for each channel
2. use reader_function to isolate the image and associate metadata from one image (from your list of choice)

In [2]:
# this will be the example for testing the pipeline below
test_img_n = TEST_IMG_N

# build the datapath
# all the imaging data goes here.
data_root_path = Path(os.path.expanduser("~")) / "Documents\Python Scripts\Infer-subc-2D"

# linearly unmixed ".czi" files are here
data_path = data_root_path / "neuron_raw_OME"
im_type = ".tiff"

# get the list of all files in "raw"
img_file_list = list_image_files(data_path,im_type)
# test_img_name = img_file_list[test_img_n]
# test_img_name

img_file_list

['C:\\Users\\Shannon\\Documents\\Python Scripts\\Infer-subc-2D\\neuron_raw_OME\\20221027_C2-107_well_1_cell_1_untreated_Linear_unmixing_decon.ome.tiff']

In [3]:
#select one image
test_img = img_file_list[0]

# isolate image as an ndarray and metadata as a dictionary
img_data, meta_dict = read_czi_image(test_img)

# # get some top-level info about the RAW data
channel_names = meta_dict['name']
img = meta_dict['metadata']['aicsimage']
scale = meta_dict['scale']
channel_axis = meta_dict['channel_axis']
huygens_meta = meta_dict['metadata']['raw_image_metadata']

c:\Users\Shannon\Anaconda3\envs\infer-subc-2D\lib\site-packages\ome_types\_convenience.py:105: FutureWarning: The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).
  d = to_dict(os.fspath(xml), parser=parser, validate=validate)


In [4]:
viewer = napari.Viewer()

In [5]:
viewer.add_image(
    img_data,
    scale=scale
)

<Image layer 'img_data' at 0x1e63a034eb0>

##  infer ***soma***

#### summary of steps

➡️ INPUT
- multi-channel sum (6.*1, 3, 2.*5)
- nuclei mask

PRE-PROCESSING
- rescaling
- denoise/smoothing
- log transform inensities
- scale to max 1.0
- create non-linear aggregate of log-intensity + scharr edge filtered intensity

CORE PROCESSING
- mask object segmentation at bottom

POST-PROCESSING
  - fill holes
  - remove small objects

OUTPUT ➡️ 
- mask of SOMA


> #### Note: this pipeline will eventually include a selection step to identify the soma that are properly labeled with all fluorescent markers. This could be one single cell per image, or more if applicable data is available.

## INPUT (prototype)

Combine multiple channels that will allow inference of the soma.

Note: the selected channels were chosen based on their qualitative ability to fill the cytoplasmic area of the cell. 

In [6]:
###################
# INPUT
###################
LYSO_CH = 3
ER_CH = 1
GOLGI_CH = 2
LIPID_CH = 0
MITO_CH = 4
PEROXI_CH = 5

struct_img_raw = (1. * img_data[LYSO_CH].copy() +
                  4. * img_data[ER_CH].copy() + 
                  1. * img_data[GOLGI_CH].copy()+
                  0. * img_data[LIPID_CH].copy()+   #because this channel labels everything, it is causing stuff outside of the the main cell to be emphasized - not great!
                  2. * img_data[MITO_CH].copy()+
                  2. * img_data[PEROXI_CH].copy())

# raw_nuclei = img_data[NUC_CH].copy() 

print(struct_img_raw.shape)
# print(raw_nuclei.shape)

(49, 1688, 1688)


### VISUALIZE: the composite image that will be used as input
Use this to adjust which channels and multiplication factors to use to create the composite input image above.

In [7]:
viewer.add_image(
    struct_img_raw,
    scale=scale
)

<Image layer 'struct_img_raw' at 0x1e63c881940>

## PRE-PROCESSING (prototype)


In [8]:

###################
# PRE_PROCESSING
###################

################# smoothing
soma_norm = min_max_intensity_normalization(struct_img_raw)


### No longer need smoothing with preprocessed images ###
# med_filter_size = 15
# soma_med = median_filter_slice_by_slice(soma_norm, 
#                                         size=med_filter_size)

# gaussian_smoothing_sigma = 1.34
# gaussian_smoothing_truncate_range = 3.0
# soma_guas = ndi.gaussian_filter(soma_med,
#                                 sigma=gaussian_smoothing_sigma,
#                                 mode="nearest", 
#                                 truncate=gaussian_smoothing_truncate_range)


################# NON-Linear aggregation
soma_log, d = log_transform(soma_norm) 
soma_log_norm = intensity_normalization(soma_log, scaling_param=[0])

# soma_edges = filters.difference_of_gaussians(soma_log_norm, 2)

# composite_soma = intensity_normalization(soma_edges, scaling_param=[0]) + soma_log_norm 

intensity normalization: min-max normalization with NO absoluteintensity upper bound


In [9]:
# viewer.add_image(
#     soma_med,
#     scale=scale
# )
# viewer.add_image(
#     soma_guas,
#     scale=scale
# )
viewer.add_image(
    soma_log_norm,
    scale=scale
)
# viewer.add_image(
#     soma_edges,
#     scale=scale
# )
# viewer.add_image(
#     composite_soma,
#     scale=scale
# )

<Image layer 'soma_log_norm' at 0x1e63c881cd0>

## CORE PROCESSING

In [10]:
###################
# CORE_PROCESSING
###################
low_level_min_size =  50

# ################# part 1
soma_binary = MO(soma_log_norm, 
                   global_thresh_method='ave', 
                   object_minArea=low_level_min_size, 
                   extra_criteria=True,
                   local_adjust= 0.05,
                   return_object=False,
                   dilate=False)                      

c:\Users\Shannon\Anaconda3\envs\infer-subc-2D\lib\site-packages\aicssegmentation\core\MO_threshold.py:52: FutureWarning: in_place argument is deprecated and will be removed in version 1.0. To avoid this warning, please do not use the in_place argument. Please see remove_small_objects documentation for more details. Please use out argument instead.
  bw_low_level = remove_small_objects(bw_low_level, min_size=object_minArea, connectivity=1, in_place=True)


In [11]:
viewer.add_image(
    soma_binary,
    scale=scale
)

<Image layer 'soma_binary' at 0x1e62b773c10>

## POST-PROCESSING

In [12]:
###################
# POST_PROCESSING
###################
hole_width = 30
removed_holes = hole_filling(soma_binary, 
                            hole_min=0, 
                            hole_max=hole_width**3,
                            fill_2d = False) 


small_object_width = 10
cleaned_img = size_filter(removed_holes, 
                          min_size= small_object_width**3, 
                          connectivity=1)

### Watershed instance segmentation is not necessary here becuase we are assuming that there is ~ one cell per image and all of the objects, even if they aren't connected are from the same cell
### For the same reason, label() is no necessary here
# watershed_mask = cleaned_img 
# inverted_img = 1. - cleaned_img

# labels_out = watershed(
#             inverted_img,
#             # markers=NU_labels,
#             connectivity= 3, # np.ones((1,3,3), bool),
#             mask=watershed_mask,
#             )

c:\Users\Shannon\Anaconda3\envs\infer-subc-2D\lib\site-packages\aicssegmentation\core\utils.py:87: FutureWarning: in_place argument is deprecated and will be removed in version 1.0. To avoid this warning, please do not use the in_place argument. Please see remove_small_objects documentation for more details. Please use out argument instead.
  return remove_small_objects(img > 0, min_size=min_size, connectivity=connectivity, in_place=False)


In [13]:
viewer.add_image(
    removed_holes,
    scale=scale
)
viewer.add_image(
    cleaned_img,
    scale=scale
)
# viewer.add_image(
#     inverted_img,
#     scale=scale
# )
# viewer.add_labels(
#     labels_out,
#     scale=scale
# )

<Image layer 'cleaned_img' at 0x1e62b7a3100>

## POST POST-PROCESSING

This process is unnecessary for neuron images, but I think its fine to keep since it doesn't do any harm

In [14]:
###################
# POST- POST_PROCESSING
###################
# keep the "SOMA" label which contains the highest total signal
all_labels = np.unique(cleaned_img)[1:]

total_signal = [soma_norm[cleaned_img == label].sum() for label in all_labels]
keep_label = all_labels[np.argmax(total_signal)]
keep_label

soma_out = np.zeros_like(cleaned_img)
soma_out[cleaned_img==keep_label] = 1

viewer.add_image(
    soma_out,
    scale=scale
)

<Image layer 'soma_out' at 0x1e62b79cd30>

## Finding the nuclei using the inverse of the cell mask

In [36]:
#################################
####### label-free nuclei #######
#################################
inverted_mask = 1 - soma_out

inverted_labels = label(inverted_mask, connectivity=1)

hole_width = 30 
removed_holes_nuc = hole_filling(inverted_mask,
                             hole_min=0, 
                             hole_max=hole_width**3, 
                             fill_2d=False)

small_object_width = 20
cleaned_img_nuc = size_filter(removed_holes_nuc, 
                          min_size= small_object_width**3,
                          method="3D",
                          connectivity=3)

all_labels_nuc = label(cleaned_img_nuc)

nuclei_label = skimage.segmentation.clear_border(cleaned_img_nuc)

c:\Users\Shannon\Anaconda3\envs\infer-subc-2D\lib\site-packages\aicssegmentation\core\utils.py:87: FutureWarning: in_place argument is deprecated and will be removed in version 1.0. To avoid this warning, please do not use the in_place argument. Please see remove_small_objects documentation for more details. Please use out argument instead.
  return remove_small_objects(img > 0, min_size=min_size, connectivity=connectivity, in_place=False)


In [37]:
# viewer.add_image(
#     inverted_mask,
#     scale=scale
# )
viewer.add_labels(
    inverted_labels,
    scale=scale
)
# viewer.add_image(
#     removed_holes_nuc,
#     scale=scale
# )
# viewer.add_image(
#     cleaned_img_nuc,
#     scale=scale
# )
# viewer.add_labels(
#     all_labels_nuc,
#     scale=scale
# )
# viewer.add_labels(
#     nuclei_label,
#     scale=scale
# )

<Labels layer 'inverted_labels' at 0x1e62cbf6820>

## Playing around with methods to isolate soma and neurites independently

In [ ]:
z_scale_ratio = scale[0]/scale[1]
spacing = (z_scale_ratio, 1, 1)

eroded = skimage.morphology.isotropic_erosion(filled, radius=10, spacing=spacing)

In [ ]:
viewer.add_image(
    eroded,
    scale=scale
)

In [ ]:
dilated = skimage.morphology.isotropic_dilation(eroded, radius=10, spacing=spacing)
viewer.add_image(
    dilated,
    scale=scale
)



<code style="background:yellow;color:black">***WIP*** 2D-->3D transition stops here</code>

## DEFINE parameterized  `_infer_soma` function

A function to infer_soma from our (Channel, 1 Z slice, X, Y) image accourding the the following parameters: 
-  

In [ ]:
def _non_linear_soma_transform_MCZ(in_img):
    """ non-linear distortion to fill out soma
    log + edge of smoothed composite
    """
    # non-Linear processing
    log_img, d = log_transform(in_img.copy()) 
    return intensity_normalization(log_img,scaling_param=[0])
    # return intensity_normalization(filters.difference_of_gaussians(log_img, 2),scaling_param=[0])  + log_img

_log_soma = _non_linear_soma_transform_MCZ(soma_guas)

viewer.add_image(
    _log_soma,
    scale=scale
)


In [ ]:
def _raw_soma_MCZ(img_in):
    """ define soma image
    """
    SOMA_W = (6.,1.,2.)
    SOMA_CH = (LYSO_CH,ER_CH,GOLGI_CH)
    img_out = np.zeros_like(img_in[0]).astype(np.double)
    for w,ch in zip(SOMA_W,SOMA_CH):
        img_out += w*img_in[ch]
    return img_out

_struct_img_raw = _raw_soma_MCZ(img_data)

viewer.add_image(
    _struct_img_raw,
    scale=scale
)


In [ ]:
def _masked_object_thresh(
    structure_img_smooth: np.ndarray, th_method: str, cutoff_size: int, th_adjust: float
) -> np.ndarray:
    """
    wrapper for applying Masked Object Thresholding with just two parameters via `MO` from `aicssegmentation`
    Parameters
    ------------
    structure_img_smooth: np.ndarray
        a 3d image
    th_method: 
         which method to use for calculating global threshold. Options include:
         "triangle" (or "tri"), "median" (or "med"), and "ave_tri_med" (or "ave").
         "ave" refers the average of "triangle" threshold and "mean" threshold.
    cutoff_size: 
        Masked Object threshold `size_min`
    th_adjust: 
        Masked Object threshold `local_adjust`

    Returns
    -------------
        np.ndimage 

    """

    struct_obj = MO(
        structure_img_smooth,
        global_thresh_method=th_method,
        object_minArea=cutoff_size,
        extra_criteria=True,
        local_adjust=th_adjust,
        return_object=False,
        dilate=True,
    )
    return struct_obj


In [ ]:
def _masked_inverted_watershed(img_in, markers, mask):
    """wrapper for watershed on inverted image and masked

    """
    labels_out = watershed(
                1. - img_in,
                markers=markers,
                connectivity=3,
                mask=mask,
                )
    return labels_out


_labels_out = _masked_inverted_watershed(cleaned_img, NU_labels, cleaned_img)

viewer.add_labels(
    _labels_out,
    scale=scale
)

In [ ]:
def _choose_max_label(raw_signal: np.ndarray, labels_in: np.ndarray):
    """ keep only the label with the maximum raw signal

    """

    all_labels = np.unique(labels_in)[1:]

    total_signal = [raw_signal[labels_in == label].sum() for label in all_labels]
    # combine NU and "labels" to make a SOMA
    keep_label = all_labels[np.argmax(total_signal)]

    labels_max = np.zeros_like(labels_in)
    labels_max[labels_in==keep_label] = 1
    return labels_max

_soma_out = _choose_max_label(soma_norm,labels_out)

viewer.add_image(
    _soma_out,
    scale=scale
)

In [ ]:
################# part 2 Nuclei pre-process
def _infer_nuclei_3D( in_img: np.ndarray,
                       median_sz: int, 
                       gauss_sig: float,
                       thresh_factor: float,
                       thresh_min: float,
                       thresh_max: float,
                       max_hole_w: int,
                       small_obj_w: int,
                       sz_filter_method: str
                     ) -> np.ndarray:
    """
    Procedure to infer 3D nuclei segmentation from multichannel z-stack input.

    Parameters
    ------------
    in_img: np.ndarray
        a 3d image containing all the channels
    soma_mask: Optional[np.ndarray] = None
        mask
    median_sz: int
        width of median filter for signal
    gauss_sig: float
        sigma for gaussian smoothing of  signal
    thresh_factor: float
        adjustment factor for log Li threholding
    thresh_min: float
        abs min threhold for log Li threholding
    thresh_max: float
        abs max threhold for log Li threholding
    max_hole_w: int
        hole filling cutoff for nuclei post-processing
    small_obj_w: int
        minimum object size cutoff for nuclei post-processing
    sz_filter_method: str
        method for size filtering; either "3D" or "slice_by_slice"

    Returns
    -------------
    nuclei_object
        mask defined extent of NU
    
    """

    nuc_ch = NUC_CH
    nuclei = select_channel_from_raw(in_img, nuc_ch)


    ###################
    # PRE_PROCESSING
    ###################                
    nuclei = min_max_intensity_normalization(nuclei)
    nuclei = median_filter_slice_by_slice(nuclei,
                                          size=median_sz)
    nuclei = image_smoothing_gaussian_slice_by_slice(nuclei,
                                                     sigma=gauss_sig )


    ###################
    # CORE_PROCESSING
    ###################
    nuclei_object = apply_log_li_threshold(nuclei, 
                                           thresh_factor=thresh_factor, 
                                           thresh_min=thresh_min, 
                                           thresh_max=thresh_max)


    ###################
    # POST_PROCESSING
    ###################
    nuclei_object = hole_filling(nuclei_object, 
                                 hole_min=0, 
                                 hole_max=max_hole_w**2, 
                                 fill_2d=True)

    nuclei_object = size_filter(nuclei_object, 
                                min_size = small_obj_w**3, 
                                method = sz_filter_method,
                                connectivity=1)


    return nuclei_object


def _fixed_infer_nuclei_3D(in_img: np.ndarray) -> np.ndarray:
    """
    Procedure to infer soma from linearly unmixed input, with a *fixed* set of parameters for each step in the procedure.  i.e. "hard coded"

    Parameters
    ------------
    in_img: np.ndarray
        a 3d image containing all the channels
    soma_mask: np.ndarray
        mask
 
    Returns
    -------------
    nuclei_object
        mask defined extent of NU
    
    """

    nuc_ch = NUC_CH
    median_sz = 4   
    gauss_sig = 1.34
    threshold_factor = 0.9
    thresh_min = 0.1
    thresh_max = 1.0
    max_hole_w = 5
    small_obj_w = 15
    sz_filter_method = "3D"

    return _infer_nuclei_3D( in_img,
                             median_sz,
                             gauss_sig,
                             threshold_factor,
                             thresh_min,
                             thresh_max,
                             max_hole_w,
                             small_obj_w,
                             sz_filter_method )


_NU_object =  _fixed_infer_nuclei_3D(img_data) 
NU_labels = label(_NU_object)

In [ ]:
##########################
# 1. infer_soma
##########################

def _infer_soma_3D(in_img: np.ndarray,
    nuclei_labels: np.ndarray,
    median_sz_soma: int,
    gauss_sig_soma: float,
    gauss_truc_rang: float,
    mo_method: str,
    mo_adjust: float,
    mo_cutoff_size: int,
    max_hole_w_soma: int,
    small_obj_w_soma: int
) -> np.ndarray:
    """
    Procedure to infer soma from linearly unmixed input.

    Parameters
    ------------
    in_img: 
        a 3d image containing all the channels
    nuclei_labels:
        a 3d mask of nuclei
    median_sz_soma: 
        width of median filter for _soma_ signal
    gauss_sig_soma: 
        sigma for gaussian smoothing of _soma_ signal
    gauss_truc_rang:
        cutoff value for gaussian
    mo_method: 
         which method to use for calculating global threshold. Options include:
         "triangle" (or "tri"), "median" (or "med"), and "ave_tri_med" (or "ave").
         "ave" refers the average of "triangle" threshold and "mean" threshold.
    mo_adjust: 
        Masked Object threshold `local_adjust`
    mo_cutoff_size: 
        Masked Object threshold `size_min`
    max_hole_w_soma: 
        hole filling cutoff for soma signal post-processing
    small_obj_w_soma: 
        minimu object size cutoff for soma signal post-processing

    Returns
    -------------
    soma_mask:
        a logical/labels object defining boundaries of soma

    """

    ###################
    # EXTRACT
    ###################
    struct_img = _raw_soma_MCZ(in_img)
    viewer.add_image(
    struct_img,
    scale=scale,
    name=1
    )


    ###################
    # PRE_PROCESSING
    ###################                         
    ################# part 1- soma
    struct_img = min_max_intensity_normalization(struct_img)
    viewer.add_image(
    struct_img,
    scale=scale,
    name=2
    )

    # make a copy for post-post processing
    scaled_signal = struct_img.copy()

    # Linear-ish processing
    struct_img = median_filter_slice_by_slice(struct_img, 
                                              size=median_sz_soma)
    viewer.add_image(
    struct_img,
    scale=scale,
    name=3
    )

    struct_img = ndi.gaussian_filter(struct_img,
                                     sigma=gauss_sig_soma,
                                     mode="nearest",
                                     truncate=gauss_truc_rang)
    viewer.add_image(
    struct_img,
    scale=scale,
    name=4
    )


    # struct_img_non_lin, d = log_transform(struct_img)
    # struct_img_non_lin = intensity_normalization(struct_img_non_lin, scaling_param=[0])
    struct_img_non_lin = _non_linear_soma_transform_MCZ(struct_img)
    viewer.add_image(
    struct_img_non_lin,
    scale=scale,
    name=5
    )
   
    ###################
    # CORE_PROCESSING
    ###################    
    struct_obj = _masked_object_thresh(struct_img_non_lin, 
                                       th_method=mo_method, 
                                       cutoff_size=mo_cutoff_size, 
                                       th_adjust=mo_adjust)               
    viewer.add_image(
    struct_obj,
    scale=scale,
    name=6
    )

    ###################
    # POST_PROCESSING
    ###################
    struct_obj = hole_filling(struct_obj, 
                              hole_min =0 , 
                              hole_max=max_hole_w_soma**2, 
                              fill_2d = True) 
    viewer.add_image(
    struct_obj,
    scale=scale,
    name=7
    )

    struct_obj = size_filter_2D(struct_obj, 
                                min_size= small_obj_w_soma**3, 
                                connectivity=1)
    viewer.add_image(
    struct_obj,
    scale=scale,
    name=8
    )

    labels_out = _masked_inverted_watershed(struct_obj, nuclei_labels, struct_obj)
    viewer.add_image(
    labels_out,
    scale=scale,
    name=9
    )

    ###################
    # POST- POST_PROCESSING
    ###################
    # keep the "SOMA" label which contains the highest total signal
    soma_out = _choose_max_label(struct_img, labels_out)

    return soma_out




## DEFINE `_fixed_infer_soma` function

Based on the _prototyping_ above define the function to infer soma. with a *fixed* set of parameters for each step in the procedure.  That is they are all "hard coded"

In [ ]:
##########################
# 1. fixed_infer_soma
##########################


def _fixed_infer_soma_3D(in_img: np.ndarray, nuclei_labels: np.ndarray) -> np.ndarray:
    """
    Procedure to infer soma from linearly unmixed input, with a *fixed* set of parameters for each step in the procedure.  i.e. "hard coded"

    Parameters
    ------------
    in_img: 
        a 3d image containing all the channels
    nuclei_object:
        a 3d mask of nuclei

    Returns
    -------------
    soma_mask:
        a logical/labels object defining boundaries of soma
    """
    

    ###################
    # PARAMETERS
    ###################   
    median_sz_soma = 15
    gauss_sig_soma = 1.34
    gauss_truc_rang = 3.0
    mo_method = "ave"
    mo_adjust = 0.15
    mo_cutoff_size = 50
    max_hole_w_soma = 100
    small_obj_w_soma = 45

    soma_out = _infer_soma_3D(in_img,
                              nuclei_labels,
                              median_sz_soma,
                              gauss_sig_soma,
                              gauss_truc_rang,
                              mo_method,
                              mo_adjust,
                              mo_cutoff_size,
                              max_hole_w_soma,
                              small_obj_w_soma) 

    return soma_out




In [ ]:

SO_label =  _fixed_infer_soma_3D(img_data, NU_labels) 

In [ ]:
viewer.add_image(
    SO_label,
    scale=scale
)

---------------------
# TEST `_infer_soma`  function defined above

<code style="background:yellow;color:black">***WIP*** 2D-->3D transition stops here</code>
##


In [ ]:
from infer_subc_2d.organelles import fixed_infer_soma, infer_soma

soma_ =  fixed_infer_soma(img_2D) 

In [ ]:
viewer.add_image(
    SO_label,
    scale=scale
)
viewer.add_image(
    soma_,
    scale=scale 
)

Write the `infer_soma` spec to the widget json

In [ ]:
from infer_subc_2d.organelles_config.helper import add_function_spec_to_widget_json

_fixed_infer_soma =  {
        "name": " infer soma mask (fixed parameters)",
        "python::module": "infer_subc_2d.organelles",
        "python::function": "fixed_infer_soma",
        "parameters": None
        }

add_function_spec_to_widget_json("fixed_infer_soma", _fixed_infer_soma, overwrite=True)

In [ ]:

_infer_soma =  {
        "name": " infer soma mask",
        "python::module": "infer_subc_2d.organelles",
        "python::function": "infer_soma",
        "parameters": {
                "median_sz_soma": {
                        "widget_type": "slider",
                        "data_type": "int",
                        "min": 3,
                        "max": 15,
                        "increment": 1
                },
                "gauss_sig_soma": {
                        "data_type": "float",
                        "increment": 0.25,
                        "max": 15.0,
                        "min": 1.25,
                        "widget_type": "slider"
                },
                "median_sz_nuc": {
                        "widget_type": "slider",
                        "data_type": "int",
                        "min": 3,
                        "max": 15,
                        "increment": 1
                },
                "gauss_sig_nuc": {
                        "data_type": "float",
                        "increment": 0.25,
                        "max": 15.0,
                        "min": 1.25,
                        "widget_type": "slider"
                },
                "mo_method": {
                        "data_type": "str",
                        "widget_type": "drop-down",
                        "options": [
                                "triangle",
                                "median",
                                "ave_tri_med"
                                ]
                },
                "mo_adjust": {
                        "data_type": "float",
                        "increment": 0.05,
                        "max": 1.0,
                        "min": 0.0,
                        "widget_type": "slider"
                },
                "mo_cutoff_size": {
                        "data_type": "int",
                        "increment": 10,
                        "max": 250,
                        "min": 10,
                        "widget_type": "slider"
                },
                "thresh_factor": {
                        "data_type": "float",
                        "increment": 0.05,
                        "max": 1.2,
                        "min": 0.6,
                        "widget_type": "slider"
                },
                "thresh_min": {
                        "data_type": "float",
                        "increment": 0.05,
                        "max": .9,
                        "min": 0.0,
                        "widget_type": "slider"
                },
                "thresh_max": {
                        "data_type": "float",
                        "increment": 0.05,
                        "max": 1.0,
                        "min": 0.1,
                        "widget_type": "slider"
                },
                "max_hole_w_nuc": {
                        "data_type": "int",
                        "increment": 1,
                        "max": 40,
                        "min": 4,
                        "widget_type": "slider"
                },           
                "small_obj_w_nuc": {
                        "data_type": "int",
                        "increment": 1,
                        "max": 50,
                        "min": 1,
                        "widget_type": "slider"
                },                           
                "max_hole_w_soma": {
                        "data_type": "int",
                        "increment": 2,
                        "max": 100,
                        "min": 20,
                        "widget_type": "slider"
                },           
                "small_obj_w_soma": {
                        "data_type": "int",
                        "increment": 1,
                        "max": 50,
                        "min": 1,
                        "widget_type": "slider"
                },        
        }
}

add_function_spec_to_widget_json("infer_soma", _infer_soma, overwrite=True )



In [ ]:

_raw_soma_MCZ =  {
        "name": "define weighted aggregate soma signal (MCZ-cellprofiler)",
        "python::module": "infer_subc_2d.organelles",
        "python::function": "raw_soma_MCZ",
        "parameters": None
        }

add_function_spec_to_widget_json("raw_soma_MCZ", _raw_soma_MCZ, overwrite=True)

In [ ]:

_non_linear_soma_transform_MCZ =  {
        "name": "non-linear filter of soma signal (MCZ-cellprofiler)",
        "python::module": "infer_subc_2d.organelles",
        "python::function": "non_linear_soma_transform_MCZ",
        "parameters": None
        }

add_function_spec_to_widget_json("non_linear_soma_transform_MCZ", _non_linear_soma_transform_MCZ)

In [ ]:

_masked_inverted_watershed =  {
        "name": "watershed on inverted image and masked",
        "python::module": "infer_subc_2d.organelles",
        "python::function": "masked_inverted_watershed",
        "parameters": None
        }

add_function_spec_to_widget_json("masked_inverted_watershed", _masked_inverted_watershed)

In [ ]:

_choose_max_label =  {
        "name": "keep only the label with the maximum raw signa",
        "python::module": "infer_subc_2d.utils.img",
        "python::function": "choose_max_label",
        "parameters": None
        }

add_function_spec_to_widget_json("choose_max_label", _choose_max_label)

In [ ]:

_min_max_intensity_normalization =  {
        "name": "Min Max Intesity Normalization",
        "python::module": "infer_subc_2d.utils.img",
        "python::function": "min_max_intensity_normalization",
        "parameters": None
        }

add_function_spec_to_widget_json("min_max_intensity_normalization", _min_max_intensity_normalization)

In [ ]:
from infer_subc_2d.organelles_config.helper import add_function_spec_to_widget_json

_masked_object_thresh =  {
        "name": "Masked Object Threshold wrapper for widgets",
        "python::module": "infer_subc_2d.utils.img",
        "python::function": "masked_object_thresh",
        "parameters": {
                "th_method": {
                        "data_type": "str",
                        "widget_type": "drop-down",
                        "options": [
                        "triangle",
                        "median",
                        "ave_tri_med"
                        ]
                },
                "cutoff_size": {
                        "data_type": "int",
                        "widget_type": "slider",
                        "min": 0,
                        "max": 2000,
                        "increment": 50
                },
                "th_adjust": {
                        "data_type": "float",
                        "widget_type": "slider",
                        "min": 0,
                        "max": 2,
                        "increment": 0.02
                }
        }
}

add_function_spec_to_widget_json("masked_object_thresh", _masked_object_thresh, overwrite=True)

--------------------------

# TEST `infer_soma` exported functions


In [ ]:
from infer_subc_2d.organelles import fixed_infer_soma

soma_mask =  fixed_infer_soma(img_2D) 

## Visualize  2


In [ ]:
# viewer = napari.Viewer()

viewer.scale_bar.visible = True

viewer.add_labels(
    soma_mask,
    scale=scale,
    blending='additive'
)


In [ ]:

from napari.utils.notebook_display import nbscreenshot

# viewer.dims.ndisplay = 3
# viewer.camera.angles = (-30, 25, 120)
nbscreenshot(viewer, canvas_only=True)
viewer.close()

-------------------------------
## Write workflow .json
Now that we've added our function specs we can compose workflows.

In [ ]:
def make_infer_soma_step_by_step_dict():
    """
    crete .json version of infer_soma
    """
    step_name = []
    function_name = []
    category =[]
    parameter_values = []
    parent = []

    ###################
    # EXTRACT
    ###################   
    # struct_img = _raw_soma_MCZ(in_img)
    step_name.append("1")
    function_name.append("raw_soma_MCZ")
    category.append("extraction")
    parameter_values.append(None)
    parent.append(0)

    step_name.append("2")
    function_name.append("select_channel_from_raw")
    category.append("extraction")
    parameter_values.append( dict(chan = NUC_CH) )
    parent.append(0)

    ###################
    # PRE_PROCESSING
    ###################
    #SOMA
    step_name.append("3")
    function_name.append("min_max_intensity_normalization")
    category.append("preprocessing")
    parameter_values.append(None)
    parent.append(1)


    step_name.append("4")
    function_name.append("median_filter_slice_by_slice")
    category.append("preprocessing")
    parameter_values.append(dict(size = 15 ))
    parent.append(3)


    step_name.append("5")
    function_name.append("image_smoothing_gaussian_slice_by_slice")
    category.append("preprocessing")
    parameter_values.append(dict( sigma = 1.4 ))
    parent.append(4)

    step_name.append("6")
    function_name.append("non_linear_soma_transform_MCZ")
    category.append("preprocessing")
    parameter_values.append(None)
    parent.append(5)

    #NUC
    step_name.append("7")
    function_name.append("min_max_intensity_normalization")
    category.append("preprocessing")
    parameter_values.append(None)
    parent.append(2)

    step_name.append("8")
    function_name.append("median_filter_slice_by_slice")
    category.append("preprocessing")
    parameter_values.append(dict(size = 4 ))
    parent.append(7)

    step_name.append("9")
    function_name.append("image_smoothing_gaussian_slice_by_slice")
    category.append("preprocessing")
    parameter_values.append(dict( sigma = 1.4 ))
    parent.append(8)

    ###################
    # CORE_PROCESSING
    ###################
    # SOMA
    step_name.append("10")
    function_name.append("masked_object_thresh")
    category.append("core")
    parameter_values.append(dict( th_method="ave_tri_med",
                                                            cutoff_size = 100,
                                                            th_adjust = 0.5))
    parent.append(6)


    # NUCLEI
    step_name.append("11")
    function_name.append("apply_log_li_threshold")
    category.append("core")
    parameter_values.append(dict(thresh_factor = 0.9, 
                                                            thresh_min = .1,
                                                            thresh_max = 1.))
    parent.append(9)


    ###################
    # POST_PROCESSING
    ###################
    # NUCLEI
    step_name.append("12")
    function_name.append("hole_filling")
    category.append("postprocessing")
    parameter_values.append(dict( hole_min=0, hole_max=5**2, fill_2d=True))
    parent.append(11)

    step_name.append("13")
    function_name.append("size_filter_2D")
    category.append("postprocessing")
    parameter_values.append(dict( min_size = 15**2  ))
    parent.append(12)

    step_name.append("14")
    function_name.append("label")
    category.append("postprocessing")
    parameter_values.append(None)
    parent.append(13)

    # SOMA
    step_name.append("15")
    function_name.append("hole_filling")
    category.append("postprocessing")
    parameter_values.append(dict( hole_min=0, hole_max=25**2, fill_2d=True))
    parent.append(10)

    step_name.append("16")
    function_name.append("size_filter_2D")
    category.append("postprocessing")
    parameter_values.append(dict( min_size = 15**2  ))
    parent.append(15)

    step_name.append("17")
    function_name.append("masked_inverted_watershed")
    category.append("postprocessing")
    parameter_values.append(None)
    parent.append([ 5 , 14, 16])

    ###################
    # POST- POST_PROCESSING
    ###################
    # keep the "SOMA" label which contains the highest total signal
    step_name.append("18")
    function_name.append("choose_max_label")
    category.append("postpostprocessing")
    parameter_values.append(None)
    parent.append([5, 17])

    ##########################
    out_dict = dict()
    for i,stepn in enumerate(step_name):
        entry = dict(category=category[i],
                            function=function_name[i],
                            parameter_values=parameter_values[i],
                            parent=parent[i]
        )
        if entry['parameter_values'] is None:
            _ = entry.pop('parameter_values')
        out_dict[stepn] = entry
        
    return out_dict

In [ ]:
from infer_subc_2d.organelles_config.helper import write_workflow_json

infer_soma_stepbystep_dict = make_infer_soma_step_by_step_dict()

write_workflow_json("conf_2.1.soma_stepbystep", infer_soma_stepbystep_dict )

In [ ]:
def make_infer_soma_step_by_step_from_raw_dict():
    """
    Procedure to infer nuclei from linearly unmixed input.

    Parameters
    ------------
    in_img: np.ndarray
        a 3d image containing all the channels

    soma_mask: np.ndarray
        mask

    Returns
    -------------
    nuclei_object
        mask defined extent of NU

    """
    step_name = []
    function_name = []
    category =[]
    parameter_values = []
    parent = []

    ###################
    # EXTRACT
    ###################   
    step_name.append("1")
    function_name.append("fixed_get_optimal_Z_img")
    category.append("extraction")
    parameter_values.append(None)
    parent.append(0)

    # SOMA
    step_name.append("2")
    function_name.append("raw_soma_MCZ")
    category.append("extraction")
    parameter_values.append(None)
    parent.append(1)

    # NUC
    step_name.append("3")
    function_name.append("select_channel_from_raw")
    category.append("extraction")
    parameter_values.append( dict(chan = NUC_CH) )
    parent.append(1)

    ###################
    # PRE_PROCESSING
    ###################
    # SOMA
    step_name.append("4")
    function_name.append("min_max_intensity_normalization")
    category.append("preprocessing")
    parameter_values.append(None)
    parent.append(2)

    step_name.append("5")
    function_name.append("median_filter_slice_by_slice")
    category.append("preprocessing")
    parameter_values.append(dict(size = 15 ))
    parent.append(4)

    step_name.append("6")
    function_name.append("image_smoothing_gaussian_slice_by_slice")
    category.append("preprocessing")
    parameter_values.append(dict( sigma = 1.4 ))
    parent.append(5)

    step_name.append("7")
    function_name.append("non_linear_soma_transform_MCZ")
    category.append("preprocessing")
    parameter_values.append(None)
    parent.append(6)

    #NUC
    step_name.append("8")
    function_name.append("min_max_intensity_normalization")
    category.append("preprocessing")
    parameter_values.append(None)
    parent.append(3)

    step_name.append("9")
    function_name.append("median_filter_slice_by_slice")
    category.append("preprocessing")
    parameter_values.append(dict(size = 4 ))
    parent.append(8)

    step_name.append("10")
    function_name.append("image_smoothing_gaussian_slice_by_slice")
    category.append("preprocessing")
    parameter_values.append(dict( sigma = 1.4 ))
    parent.append(9)

    ###################
    # CORE_PROCESSING
    ###################
    # SOMA
    step_name.append("11")
    function_name.append("masked_object_thresh")
    category.append("core")
    parameter_values.append(dict( th_method="ave_tri_med",
                                                            cutoff_size = 100,
                                                            th_adjust = 0.5))
    parent.append(7)

    # NUC
    step_name.append("12")
    function_name.append("apply_log_li_threshold")
    category.append("core")
    parameter_values.append(dict(thresh_factor = 0.9, 
                                                            thresh_min = .1,
                                                            thresh_max = 1.))
    parent.append(10)

    ###################
    # POST_PROCESSING
    ###################
    # nUC
    step_name.append("13")
    function_name.append("hole_filling")
    category.append("postprocessing")
    parameter_values.append(dict( hole_min=0, hole_max=5**2, fill_2d=True))
    parent.append(12)

    step_name.append("14")
    function_name.append("size_filter_2D")
    category.append("postprocessing")
    parameter_values.append(dict( min_size = 15**2  ))
    parent.append(13)

    step_name.append("15")
    function_name.append("label")
    category.append("postprocessing")
    parameter_values.append(None)
    parent.append(14)


    # SOMA
    step_name.append("16")
    function_name.append("hole_filling")
    category.append("postprocessing")
    parameter_values.append(dict( hole_min=0, hole_max=25**2, fill_2d=True))
    parent.append(11)

    step_name.append("17")
    function_name.append("size_filter_2D")
    category.append("postprocessing")
    parameter_values.append(dict( min_size = 15**2  ))
    parent.append(16)

    step_name.append("18")
    function_name.append("masked_inverted_watershed")
    category.append("postprocessing")
    parameter_values.append(None)
    parent.append([  7 , 15,17 ])

    ###################
    # POST- POST_PROCESSING
    ###################
    # keep the "SOMA" label which contains the highest total signal
    step_name.append("19")
    function_name.append("choose_max_label")
    category.append("postpostprocessing")
    parameter_values.append(None)
    parent.append([6,18])

    ##########################
    out_dict = dict()
    for i,stepn in enumerate(step_name):
        entry = dict(category=category[i],
                            function=function_name[i],
                            parameter_values=parameter_values[i],
                            parent=parent[i]
        )
        if entry['parameter_values'] is None:
            _ = entry.pop('parameter_values')
        out_dict[stepn] = entry
        
    return out_dict

In [ ]:

infer_soma_stepbystep_from_raw_dict = make_infer_soma_step_by_step_from_raw_dict()

write_workflow_json("conf_1.1.soma_stepbystep_from_raw", infer_soma_stepbystep_from_raw_dict)

-------------
## SUMMARY

The above explains the overall framework.  

### NEXT: INFER NUCLEI

proceed to [02_infer_nuclei.ipynb](./02_infer_nuclei.ipynb)
